In [4]:
import os
import requests
import pandas as pd

In [20]:
def fetch_kaiko_data(api_key, base_assets, quote_asset, start_time, end_time):
    base_url = "https://us.market-api.kaiko.io/v2/data/trades.v1/spot_direct_exchange_rate"
    headers = {
        'Accept': 'application/json',
        'X-Api-Key': api_key
    }
    
    data_frames = []

    for base in base_assets:
        quote = 'usd'
        endpoint_url = f"{base_url}/{base}/{quote_asset}"
        params = {
            "start_time": start_time,
            "end_time": end_time,
            "interval": "1d",
            "page_size": 1000  # you can adjust this based on your needs
        }
        
        response = requests.get(endpoint_url, headers=headers, params=params)
        
        if response.status_code == 200:
            data = response.json()["data"]
            df = pd.DataFrame(data)
            df['asset'] = base + '-' + quote
            data_frames.append(df)
        else:
            print(f"Failed to fetch data for {base}. Status Code: {response.status_code}")
            print(response.text)
            
    final_df = pd.concat(data_frames, ignore_index=True)
    return final_df

def check_missing_values(df):
    missing = df.isnull().sum()
    columns_with_missing = missing[missing > 0]
    
    if columns_with_missing.empty:
        print("All columns have values for all dates.")
    else:
        print("Columns with missing values:")
        print(columns_with_missing)

In [29]:
API_KEY = os.environ.get('KAIKO_API_KEY')
BASE_ASSETS = ['bch', 'eth', 'xrp', 'ltc', 'dot']
QUOTE_ASSET = 'usd'
START_TIME = "2021-01-03T00:00:00Z"
END_TIME = "2023-09-15T23:59:59Z"

df = fetch_kaiko_data(API_KEY, BASE_ASSETS, QUOTE_ASSET, START_TIME, END_TIME)
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df_pivot = df.pivot(index='timestamp', columns='asset', values='price')
df_pivot.sort_index(inplace=True)

# Convert the object columns to float
for column in df_pivot.columns:
    df_pivot[column] = pd.to_numeric(df_pivot[column], errors='coerce')

# Check datatypes again to confirm conversion
df_pivot.dtypes


asset
bch-usd    float64
dot-usd    float64
eth-usd    float64
ltc-usd    float64
xrp-usd    float64
dtype: object

In [30]:
check_missing_values(df_pivot)

All columns have values for all dates.


In [31]:
df_pivot.head()

asset,bch-usd,dot-usd,eth-usd,ltc-usd,xrp-usd
timestamp,,,,,
2021-01-03,391.500716,9.579936,903.979183,149.923097,0.224541
2021-01-04,416.109502,9.634091,1013.921548,155.891473,0.234463
2021-01-05,409.310649,9.662070,1063.086335,155.204442,0.229103
2021-01-06,436.851261,9.998653,1135.564878,163.367945,0.242095
2021-01-07,454.574195,10.036309,1222.697987,170.912772,0.315176


In [41]:
# Proceed with the index calculation

# 1. Calculate the initial average of all 5 assets
initial_average = df_pivot.iloc[0].mean()

# 2. Set the initial index level
initial_index_level = 1000

# 3. Calculate the divisor
divisor = initial_average / initial_index_level

# 1. Calculate the average for each date
df_pivot['average_price'] = df_pivot.mean(axis=1)

# 2. Calculate the index level for each date
df_pivot['index_level'] = df_pivot['average_price'] / divisor

df_pivot


asset,bch-usd,dot-usd,eth-usd,ltc-usd,xrp-usd,average_price,index_level
timestamp,,,,,,,
2021-01-03,391.500716,9.579936,903.979183,149.923097,0.224541,406.789822,1000.000000
2021-01-04,416.109502,9.634091,1013.921548,155.891473,0.234463,446.088671,1096.607258
2021-01-05,409.310649,9.662070,1063.086335,155.204442,0.229103,457.745947,1125.264013
2021-01-06,436.851261,9.998653,1135.564878,163.367945,0.242095,488.085131,1199.845977
2021-01-07,454.574195,10.036309,1222.697987,170.912772,0.315176,519.536713,1277.162517
...,...,...,...,...,...,...,...
2023-09-11,185.952721,4.030275,1571.496608,59.718355,0.478154,509.232749,1251.832572
2023-09-12,199.932053,4.005192,1592.560012,60.252475,0.477658,519.170780,1276.262956
2023-09-13,200.576690,4.004805,1599.978105,61.327061,0.480130,521.725616,1282.543435
